<a href="https://colab.research.google.com/github/DhayuIntan/Machine_Learning/blob/main/jobsheet_10/praktikum_02_tugas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Praktikum 2**
## Generator Teks dengan RNN

## **Setup**

### **Import TensorFlow**

In [2]:
import tensorflow as tf
import numpy as np
import os
import time

### **Download Dataset Shakespeare**

In [3]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


### **Load Data**

In [4]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [5]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [6]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


## **Olah Teks**

### **Vectorize Teks**
tf.keras.layers.StringLookup dapat mengubah setiap karakter menjadi ID numerik. Caranya adalah teks akan dipecah menjadi token terlebih dahulu.

In [7]:
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

sekarang buat tf.keras.layers.StringLookup layer:

In [8]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab),
    mask_token=None
)

perintah diatas mengconvert token menjadi id

In [9]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [10]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

Lapisan ini mengconvert kembali karakter dari vektor ID, dan mengembalikannya sebagai karakter tf.RaggedTensor:

In [11]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Gunakan tf.strings.reduce_join untuk menggabungkan kembali karakter menjadi string.

In [12]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [13]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### **Prediksi**

Diberikan sebuah karakter, atau serangkaian karakter, karakter apa yang paling mungkin berikutnya? Ini adalah tugas yang harus Anda latih agar model dapat melakukannya. Masukan ke model akan berupa urutan karakter, dan Anda melatih model untuk memprediksi keluaran berupa karakter berikut pada setiap langkah waktu. Karena RNN mempertahankan keadaan internal yang bergantung pada elemen yang terlihat sebelumnya, dan mengingat semua karakter dihitung hingga saat ini.

### **Membuat Trianing Set dan Target**
Gunakan fungsi tf.data.Dataset.from_tensor_slices untuk mengonversi vektor teks menjadi aliran indeks karakter.

In [14]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [16]:
 for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [17]:
seq_length = 100

Metode batch memungkinkan Anda dengan mudah mengonversi karakter individual ini menjadi urutan ukuran yang diinginkan.

In [18]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


akan lebih mudah untuk melihat apa yang dilakukan jika Anda menggabungkan token kembali menjadi string:

In [19]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


### **Membuat Batch Training**

In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## **Buat Model**

Model memiliki 3 lapisan neural network :


*   tf.keras.layers.Embedding: Lapisan masukan. Tabel pencarian yang dapat dilatih yang akan memetakan setiap karakter-ID ke vektor dengan dimensi embedding_dim;
*   tf.keras.layers.GRU: lapisan RNN dengan ukuran unit=rnn_units (Anda juga dapat menggunakan lapisan LSTM di sini.)
* tf.keras.layers.Dense: Lapisan keluaran, dengan keluaran vocab_size. Ini menghasilkan satu logit untuk setiap karakter dalam kosakata. Ini adalah log kemungkinan setiap karakter menurut model.



In [25]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [26]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [27]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

## **Uji Model**

In [28]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [29]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  16896     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [30]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

Pada setiap langkah waktu, prediksi indeks karakter berikutnya:

In [31]:
sampled_indices

array([28,  3, 63, 33, 39, 50, 27, 51, 38, 21, 45, 30, 55,  3, 54, 58,  9,
       64, 46, 42, 34, 20, 28, 16, 36, 43,  6, 21, 47, 46, 40, 17,  2, 53,
       26,  4, 49, 59,  7, 41,  6, 32, 45,  5, 34, 30, 12, 37, 46, 20,  6,
       52, 48,  3, 13, 40, 34,  7, 24, 29,  5, 43,  4, 40,  1, 47, 44, 20,
       12,  5, 24,  3, 46, 36, 47, 55, 17, 17, 39, 46, 28, 47,  5, 36,  8,
       29, 58, 23, 14, 28, 11, 38, 61, 10, 60, 24, 13, 12,  8, 64])

Dekode kode berikut untuk melihat teks yang diprediksi oleh model tidak terlatih ini:

In [32]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'your wit restored!\nO, that once more you knew but what you are!\nThese fifteen years you have been in'

Next Char Predictions:
 b"O!xTZkNlYHfQp!os.ygcUGOCWd'HhgaD nM$jt,b'Sf&UQ;XgG'mi!?aU,KP&d$a\nheG;&K!gWhpDDZgOh&W-PsJAO:Yv3uK?;-y"


## **Train Model**

### **Tambahkan optimizer dan fungsi loss**

In [33]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [34]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1901174, shape=(), dtype=float32)


Periksa apakah eksponensial dari loss rata-rata kira-kira sama dengan ukuran kosakata. Loss yang jauh lebih tinggi berarti model tersebut yakin akan jawaban yang salah, dan memiliki inisialisasi yang buruk:

In [35]:
tf.exp(example_batch_mean_loss).numpy()

66.03054

Konfigurasikan prosedur pelatihan menggunakan metode tf.keras.Model.compile. Gunakan tf.keras.optimizers.Adam dengan argumen default dan fungsi loss.

In [36]:
model.compile(optimizer='adam', loss=loss)

### **Konfigurasi Checkpoints**

Gunakan tf.keras.callbacks.ModelCheckpoint untuk memastikan bahwa checkpoint disimpan selama pelatihan:

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### **Lakukan Proses Training**

In [38]:
EPOCHS = 20

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 14s 55ms/step - loss: 2.7063
Epoch 2/20
172/172 [==============================] - 11s 53ms/step - loss: 1.9859
Epoch 3/20
172/172 [==============================] - 11s 56ms/step - loss: 1.7156
Epoch 4/20
172/172 [==============================] - 12s 55ms/step - loss: 1.5570
Epoch 5/20
172/172 [==============================] - 12s 59ms/step - loss: 1.4577
Epoch 6/20
172/172 [==============================] - 15s 62ms/step - loss: 1.3898
Epoch 7/20
172/172 [==============================] - 15s 62ms/step - loss: 1.3368
Epoch 8/20
172/172 [==============================] - 14s 60ms/step - loss: 1.2918
Epoch 9/20
172/172 [==============================] - 12s 60ms/step - loss: 1.2520
Epoch 10/20
172/172 [==============================] - 12s 58ms/step - loss: 1.2127
Epoch 11/20
172/172 [==============================] - 14s 60ms/step - loss: 1.1739
Epoch 12/20
172/172 [==============================] - 13s 59ms/step - loss: 1.1349
E

Membuat prediksi satu langkah:

In [40]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [41]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [42]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
Even art thou years? his son shot arms,
If thy old consent all my needing, have embiding
Romeo, what dont their lots this hard, and
to differeign might frank, and humour'd thoughts, slaughters,
His wealth of Pompery.

MARCIUS:
Do remain, as well as herself, beseech you any
That we 'njuct me force. But what saundst thou not?

LUCENTIO:
Didst perife
With resemfting in that sin
To move our deputy.

VINCENTIO:
What is the model whereof, if it do not
Unsturbuh thee this blushes and bran himself,
Stulls for Edward: but thou sett'st manifest?

BATTONSOP:
I ne'er a sword, for how Juliet.

Clown:
He seems to make this late: little love,
Sweat love it. Heaven meet thee protect him I will put demand.
3 KING HENRY VI

KING HENRY VI:
Hadst thou pluck my fame and very suncher's demand?
Yourself, as thou diest true; thou'rt in rest,
And sigh'd false Edmiode, steal it, and came
That e'er I come, in't speaking, wast whoreshapts and mode?
Hadst thou not shed the good gods behind you; York and Eng

In [43]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
These secessions women once great Henry, royal grand;
And thou nog at the point.

GLOUCESTER:
Love with him to't.

PRINCE:
We sea, if it do, I am false for: but do not slow,
I'll voice us and now by himself.

DUKE VINCENTIO:
I'll pray thee, As the truth I have to understand myself:
A servant granted and run bastards, and my revenue intitle,
with tears was't therein and sparl to gift me Henry.

CLIFFORD:
And here I leave, I would say, knock advices
of noble pursuing; and I choose hang her;
And thou the lie-giver penutoo, he
makes fears so well, To-morrow to your meetinguars!

YORK:
I would be subjects to a keeper. A beggar studied
I have at one and a penitent poor.

Page:
Sirrah, hold thy unharphus-and an old man's milk at midning.

SAMPSON:
Fear none, and, bear, her cunning.

MARCIUS:
Then, tell me,'t, a stirrior, master, leave the
spept wax of justice,' and that willing to
considered, then, adieu;
Go; tell us rich, then I pretition
These diversion will show you mercy;
For he ha

## **Ekspor Model Generator**

In [44]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [45]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Katharina,
An you but fair, by witness, shall at you much disglades;
They were these nature's frown


# **Tugas**

In [46]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
    inputs, labels = inputs
    with tf.GradientTape() as tape:
      predictions = self(inputs, training=True)
      loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [47]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

In [48]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
)

In [49]:
model.fit(dataset, epochs=1)

172/172 [==============================] - 14s 61ms/step - loss: 2.6806


In [50]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.1823
Epoch 1 Batch 50 Loss 2.0557
Epoch 1 Batch 100 Loss 1.9586
Epoch 1 Batch 150 Loss 1.8436

Epoch 1 Loss: 1.9692
Time taken for 1 epoch 13.16 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.8022
Epoch 2 Batch 50 Loss 1.7794
Epoch 2 Batch 100 Loss 1.6933
Epoch 2 Batch 150 Loss 1.6201

Epoch 2 Loss: 1.6981
Time taken for 1 epoch 11.53 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5602
Epoch 3 Batch 50 Loss 1.5981
Epoch 3 Batch 100 Loss 1.4975
Epoch 3 Batch 150 Loss 1.5190

Epoch 3 Loss: 1.5435
Time taken for 1 epoch 11.28 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4703
Epoch 4 Batch 50 Loss 1.4452
Epoch 4 Batch 100 Loss 1.4101
Epoch 4 Batch 150 Loss 1.4484

Epoch 4 Loss: 1.4468
Time taken for 1 epoch 11.22 sec
_____________________________________________________________________

# **SOAL**
Jalankan kode diatas dan sebutkan perbedaanya dengan praktikum 2?

# **JAWABAN**

Pada praktikum, proses yang dilakukan lebih sederhana yaitu dengan otomisasi menggunakan fungsi 'model.fit()', sedangkan pada tugas digunakan custom training loop sehingga kita dapat mengatur langkah-langkah pelatihan, seperti perhitungan gradien, optimasi, dan pelaporan. Pada tugas digunakan metode 'train_step' yang dapat menghitung gradien loss terhadap parameter model untuk digunakan dalam mengupdate parameter model. Selain itu, pada tugas digunakan objek tf.metrics.Mean() untuk memantau rata-rata loss selama pelatihan. Kemudian, pada setiap batch, rata-rata loss diperbarui dengan memanggil mean.update_state(logs['loss']) sehingga memungkinkan kita untuk melacak perubahan loss secara dinamis selama pelatihan. Secara umum, pada kode tugas, kita memiliki lebih banyak kontrol untuk memantau serta mengelola proses pelatihan yang sesuai.